### NGC API KEY 

In [1]:
# Replace MY_NGC_API_KEY with your generated NGC API key
api_key = "ZDBvdnM4Y2RwZG5oNjBzMnVhOThsMm84OTE6OTJkODM3MDgtMmFmNi00ODk5LWIyOGUtMDg2ZWMzNjQwN2Ri"

In [2]:
!docker login -u '$oauthtoken' -p $api_key nvcr.io

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/mjk0307/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### Installing triton

In [3]:
!docker pull nvcr.io/nvidia/tritonserver:22.11-py3

22.11-py3: Pulling from nvidia/tritonserver
Digest: sha256:1cb91225af75736bf358e2bbd5892defcf03692115bd6bf4ec31e17678426e8f
Status: Image is up to date for nvcr.io/nvidia/tritonserver:22.11-py3
nvcr.io/nvidia/tritonserver:22.11-py3


In [25]:
# 서버에서 아래 복사 및 붙여넣기 

docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v /home/mjk0307/nemo-text-classification/onnx_model:/models "nvcr.io/nvidia/tritonserver:22.11-py3" tritonserver --model-repository=/models
            

SyntaxError: invalid syntax (2523748184.py, line 3)

In [3]:
!curl -v localhost:8000/v2/health/ready

*   Trying 127.0.0.1:8000...
* TCP_NODELAY set
* connect to 127.0.0.1 port 8000 failed: Connection refused
* Failed to connect to localhost port 8000: Connection refused
* Closing connection 0
curl: (7) Failed to connect to localhost port 8000: Connection refused


### triton inference

In [24]:
import argparse
import numpy as np
import sys
from functools import partial
import os
import tritongrpcclient
import tritongrpcclient.model_config_pb2 as mc
import tritonhttpclient
from tritonclientutils import triton_to_np_dtype
from tritonclientutils import InferenceServerException
from transformers import AutoTokenizer
from scipy.special import softmax

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
VERBOSE = False
input_name = ['input_ids', 'attention_mask', 'token_type_ids']
output_name = 'logits'


def run_inference(text, model_name='kcelectra', url='127.0.0.1:8000', model_version='2'):
    triton_client = tritonhttpclient.InferenceServerClient(
        url=url, verbose=VERBOSE)
    model_metadata = triton_client.get_model_metadata(
        model_name=model_name, model_version=model_version)
    model_config = triton_client.get_model_config(
        model_name=model_name, model_version=model_version)
    
    inputs = tokenizer.batch_encode_plus([text], max_length=128, truncation=True, padding='max_length')
    input_ids, token_type_ids, attention_mask = inputs.values()
    input_ids = np.array(input_ids, dtype=np.int64)
    attention_mask = np.array(attention_mask, dtype=np.int64)
    token_type_ids = np.array(token_type_ids, dtype=np.int64)
  
    input_ids = input_ids.reshape(1,128)
    attention_mask = attention_mask.reshape(1,128)
    token_type_ids = token_type_ids.reshape(1,128)
    
    print(input_ids)
    print(attention_mask)
    print(token_type_ids)
    
    input0 = tritonhttpclient.InferInput(input_name[0], (1, 128), 'INT64')
    input0.set_data_from_numpy(input_ids, binary_data=False)
    input1 = tritonhttpclient.InferInput(input_name[1], (1, 128), 'INT64')
    input1.set_data_from_numpy(attention_mask, binary_data=False)
    input2 = tritonhttpclient.InferInput(input_name[2], (1,128), 'INT64')
    input2.set_data_from_numpy(token_type_ids, binary_data=False)
    
    
    output = tritonhttpclient.InferRequestedOutput(output_name,  binary_data=False)
    response = triton_client.infer(model_name,model_version=model_version, inputs=[input0, input1, input2], outputs=[output])
    logits = response.as_numpy('logits')
    logits = np.asarray(logits, dtype=np.float32)
    
    print(logits)
    
#     # we throw away "neutral" (dim 1) and take the probability of
#     # "entailment" (2) as the probability of the label being true 
#     entail_contradiction_logits = logits[:,[0,2]]
#     probs = softmax(entail_contradiction_logits)
#     true_prob = probs[:,1].item() * 100
#     print(f'Probability that the label is true: {true_prob:0.2f}%')
    
    
    
# topic classification premises
if __name__ == '__main__':
    run_inference("오늘 날씨 죽인다")

[[    2  8606 19098 12209  4058     3     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]
[[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

### example to features

In [26]:
test_data=open('data/auc/test.tsv').readlines()

In [58]:
texts=[]
labels=[]
for line in test_data:
    try:
        text, label = line.split('\t')
        texts.append(text)
        labels.append(label)
    except:
        continue

In [42]:
inputs=[]
for text in texts:
    inputs.append(tokenizer.batch_encode_plus(
        [text],
        max_length=128,
        padding="max_length",
        truncation=True))

In [43]:
len(inputs)

1978

In [45]:
inputs[1200]

{'input_ids': [[2, 2470, 11030, 29724, 4082, 202, 8804, 19612, 1568, 8920, 13618, 4422, 13618, 4065, 2821, 9071, 4071, 8108, 8736, 20442, 8147, 12889, 16360, 8306, 8144, 8057, 13618, 4065, 12904, 9685, 4071, 2741, 13273, 9097, 8427, 8102, 640, 20950, 7973, 8622, 4331, 1624, 9653, 4378, 4053, 31423, 4115, 17633, 9455, 11395, 4071, 22569, 3537, 4172, 12741, 9455, 8270, 9693, 8241, 640, 8403, 9653, 15668, 30621, 4115, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [46]:
import numpy as np
length=[]

for text in texts:
    length.append(len(tokenizer.tokenize(text)))
length=np.array(length)
length.mean() #데이터의 평균 토큰 길이

75.14964610717897

In [70]:
from torch.utils.data import TensorDataset
from torch.utils.data import SequentialSampler, DataLoader
import torch

In [66]:
all_input_ids=torch.tensor([i.input_ids for i in inputs], dtype=torch.long)
all_token_type_ids=torch.tensor([i.token_type_ids for i in inputs], dtype=torch.long)
all_attention_mask=torch.tensor([i.attention_mask for i in inputs], dtype=torch.long)

In [59]:
labels=[l.strip() for l in labels]

In [61]:
labels=[int(l) for l in labels]

In [62]:
all_labels=torch.tensor(labels)

In [69]:
dataset=TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [72]:
# dataloader 정의

results = {}
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=16)

In [73]:
from sklearn import metrics as sklearn_metrics

In [74]:
def simple_accuracy(labels, preds):
    return (labels == preds).mean()

In [151]:
import argparse
import numpy as np
import sys
from functools import partial
import os
import tritongrpcclient
import tritongrpcclient.model_config_pb2 as mc
import tritonhttpclient
from tritonclientutils import triton_to_np_dtype
from tritonclientutils import InferenceServerException
from transformers import AutoTokenizer
from scipy.special import softmax

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
VERBOSE = False
input_name = ['input_ids', 'attention_mask', 'token_type_ids']
output_name = 'logits'


def run_inference(batch, model_name='kcelectra', url='127.0.0.1:8000', model_version='2'):
    triton_client = tritonhttpclient.InferenceServerClient(
        url=url, verbose=VERBOSE)
    model_metadata = triton_client.get_model_metadata(
        model_name=model_name, model_version=model_version)
    model_config = triton_client.get_model_config(
        model_name=model_name, model_version=model_version)
    
    #print(batch)
    #inputs = tokenizer.batch_encode_plus(batch, max_length=128, truncation=True, padding='max_length')
    input_ids= batch["input_ids"]
    attention_mask=batch["attention_mask"]
    token_type_ids=batch["token_type_ids"]
    input_ids = np.array(input_ids, dtype=np.int64)
    attention_mask = np.array(attention_mask, dtype=np.int64)
    token_type_ids = np.array(token_type_ids, dtype=np.int64)

    num=len(input_ids)
    input_ids = input_ids.reshape(num,128)
    attention_mask = attention_mask.reshape(num,128)
    token_type_ids = token_type_ids.reshape(num,128)


    input0 = tritonhttpclient.InferInput(input_name[0], (num, 128), 'INT64')
    input0.set_data_from_numpy(input_ids, binary_data=False)
    input1 = tritonhttpclient.InferInput(input_name[1], (num, 128), 'INT64')
    input1.set_data_from_numpy(attention_mask, binary_data=False)
    input2 = tritonhttpclient.InferInput(input_name[2], (num,128), 'INT64')
    input2.set_data_from_numpy(token_type_ids, binary_data=False)

    
    output = tritonhttpclient.InferRequestedOutput(output_name,  binary_data=False)
    response = triton_client.infer(model_name,model_version=model_version, inputs=[input0, input1, input2], outputs=[output])
    logits = response.as_numpy('logits')
    logits = np.array(logits, dtype=np.float32)
    
    logits_all.append(logits)

In [166]:
import time
import datetime

logits_all=[]
labels_all=[]
start=time.time()
for batch in eval_dataloader:
    with torch.no_grad():
        inputs={"input_ids":batch[0], "attention_mask":batch[1], "token_type_ids":batch[2], "labels":batch[3]}
        run_inference(inputs)
        labels_all.append(inputs["labels"].detach().cpu().numpy())
sec=time.time()-start
print(str(datetime.timedelta(seconds=sec)))

0:00:01.925930


In [156]:
logits_all

[array([[-5.4609375, -4.1835938, 14.171875 , -6.6328125, -3.4414062],
        [13.640625 , -6.2148438, -4.0429688, -6.6914062, -4.515625 ],
        [-5.9453125, -3.5410156, 13.84375  , -6.4453125, -3.5019531],
        [-5.4492188, -4.015625 , 14.1796875, -6.640625 , -3.6074219],
        [14.21875  , -6.53125  , -4.5351562, -7.0078125, -4.4492188],
        [14.2421875, -6.4960938, -4.6640625, -7.1679688, -4.3125   ],
        [-5.6210938, -3.9609375, 14.1015625, -6.6640625, -3.40625  ],
        [14.1953125, -6.46875  , -4.5625   , -7.0546875, -4.421875 ],
        [14.2421875, -6.4765625, -4.8046875, -7.1601562, -4.2148438],
        [13.9296875, -6.2460938, -4.7304688, -7.0664062, -4.1367188],
        [-5.8476562, -3.6601562, 13.9609375, -6.6640625, -3.3808594],
        [-4.953125 , -4.5625   , 14.1484375, -6.734375 , -3.3847656],
        [14.0625   , -6.3125   , -4.953125 , -7.03125  , -4.125    ],
        [-6.0117188, -3.1445312, 13.7265625, -6.5507812, -3.59375  ],
        [14.0546875,

In [157]:
labels_all

[array([2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2]),
 array([2, 2, 2, 2, 0, 3, 0, 0, 0, 2, 2, 4, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4]),
 array([4, 4, 0, 0, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4]),
 array([4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2]),
 array([2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]),
 array([2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 array([4, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4]),
 array([4, 4, 4, 3, 0, 3, 3, 3, 0, 0, 0, 3, 4, 2, 2, 2]),
 array([2, 4, 

In [158]:
preds=[]
for batch in logits_all:
    preds.extend(np.argmax(batch, axis=1))
    
real=[]
for batch in labels_all:
    real.extend([i for i in batch])

In [159]:
print(preds)

[2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 1, 1, 1, 1, 1, 0, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 4, 4, 4, 4, 4, 1, 1, 3, 1, 3, 1, 2, 1, 3, 4, 4, 4, 4, 4, 4, 4, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 4, 2, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 0, 3, 3, 3, 0, 0, 0, 3, 4, 2, 2, 2, 2, 4, 3, 4, 0, 0, 0, 0, 2, 3, 3, 3, 4, 4, 0, 0, 3, 4, 2, 1, 4, 1, 3, 1, 3, 1, 3, 4, 4, 1, 4, 4, 1, 1, 3, 4, 0, 2, 2, 2, 1, 4, 2, 3, 3, 3, 4, 4, 2, 2, 4, 1, 4, 1, 1, 1, 1, 3, 1, 0, 0, 

In [160]:
print(real)

[2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 3, 0, 0, 0, 2, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 0, 3, 3, 3, 0, 0, 0, 3, 4, 2, 2, 2, 2, 4, 3, 4, 0, 0, 0, 0, 2, 3, 3, 3, 4, 4, 4, 0, 3, 4, 2, 2, 4, 1, 3, 1, 3, 1, 3, 4, 4, 1, 4, 4, 1, 1, 3, 4, 2, 2, 2, 4, 1, 1, 2, 3, 3, 3, 4, 4, 2, 2, 4, 1, 1, 1, 1, 1, 1, 3, 1, 0, 0, 

In [165]:
result={
        "precision": sklearn_metrics.precision_score(real, preds, average="macro"),
        "recall": sklearn_metrics.recall_score(real, preds, average="macro"),
        "f1": sklearn_metrics.f1_score(real, preds, average="macro"),
        "f1_weighted": sklearn_metrics.f1_score(real, preds, average="weighted"),
        "acc": simple_accuracy(np.array(real), np.array(preds))
}

result

{'precision': 0.8834818271410247,
 'recall': 0.8846228055261074,
 'f1': 0.8830914579506081,
 'f1_weighted': 0.8830412715622392,
 'acc': 0.8832153690596563}